In [1]:
import os
import psycopg2
import csv
# import pandas as pd 

In [2]:
# conectar banco de dados
USERNAME = os.environ.get('DB_USERNAME')
PASSWORD = os.environ.get('DB_PASSWORD')
DB_NOME = os.environ.get('DB_NAME')


In [7]:
ano = "2022"
arquivo_nome = f"chamada_regular_sisu_{ano}_1"
arquivo_path = f'./sisu/{ano}/{arquivo_nome}.csv'
nome_tabela = arquivo_nome.replace('sisu_', '')

In [4]:
# criar tabela no banco de dados
# from sqlalchemy import create_engine 
# data = pd.read_csv(arquivo_path, sep='|', encoding='ISO-8859-1')
# lento e cria tabelas automaticamente. útil para datasets pequenos
# data.to_sql(nome_tabela, engine, if_exists='replace', index=False)
# engine.dispose()

In [17]:
 
conn = psycopg2.connect(database=DB_NOME, user=USERNAME, password=PASSWORD, host="localhost", port="5432")
cur = conn.cursor()

# tabela tem que ser criada antes

## criar tabela | ISO-8859-1
# with open(arquivo_path, 'r', encoding='ISO-8859-1') as f:
#     reader = csv.reader(f, delimiter='|')
#     next(reader)  
#     cur.copy_expert(f"COPY {nome_tabela} FROM STDIN CSV DELIMITER '|' NULL AS ''", f)

## criar tabela ; utf-8
with open(arquivo_path, 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=';')
    next(reader)  
    cur.copy_expert(f"COPY {nome_tabela} FROM STDIN CSV DELIMITER ';' NULL AS ''", f)


conn.commit()
conn.close()